In [1]:
import os 

In [3]:
os.chdir("../")

In [36]:
%pwd

'/home/mohammadrasel/project/another/Kidney-Disease-Classification-Deep-Learning-Project'

In [54]:



import dagshub
dagshub.init(repo_owner='mrhrasel232', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "mrhrasel232"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "5eca32a19c9c018662f747dd2998fd14cb1c053e"

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

[2026-02-15 23:06:20,402: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project"

[2026-02-15 23:06:20,406: INFO: helpers: Initialized MLflow to track repo "mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2026-02-15 23:06:20,408: INFO: helpers: Repository mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project initialized!]


In [45]:
import tensorflow as tf

In [47]:
model=tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [46]:
print(model)

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    
    params_image_size: list 
    params_batch_size: int 


In [49]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [50]:
class ConfigurationManager:
    def __init__(
        self,config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) ->EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model="artifacts/training/trained_model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/mrhrasel232/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config

In [51]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [52]:
class Evaluation: 
    def __init__(self, config: EvaluationConfig):
        self.config=config 
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores={"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("source.json"), data= scores)

    def log_into_mlfow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
        
            

    

In [56]:
try:
    config=ConfigurationManager()
    evaluation_config=config.get_evaluation_config()
    evaluation=Evaluation(evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlfow()
except Exception as e:
    raise e

[2026-02-15 23:11:12,263: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-15 23:11:12,266: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-15 23:11:12,267: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.


2026-02-15 23:11:13.181437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


9/9 [==============================] - 35s 4s/step - loss: 15.9428 - accuracy: 0.5180
[2026-02-15 23:11:47,722: INFO: common: json file saved at: source.json]


2026/02/15 23:11:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-02-15 23:11:51,354: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpdhb1k02p/model/data/model/assets
[2026-02-15 23:11:52,059: INFO: builder_impl: Assets written to: /tmp/tmpdhb1k02p/model/data/model/assets]


KeyboardInterrupt: 